In [ ]:
from broprompt import Prompt, parse_codeblock_to_dict, get_yaml_schema_definition
from brollm import BedrockChat
from pydantic import BaseModel, Field
model = BedrockChat()

In [2]:
so_prompt = Prompt.from_markdown("./structured_output.md")
print(so_prompt.str)

# Information Extractor  

# Persona  
You're an expert on extracting information from given INPUT.  

# Instruction  
- read INPUT carefully and extract information based on SCHEMAS.  

# Caution
- return only specified schemas.  
- no conversation or chit-chat is needed  

# Data Format  
Return the extracted information in YAML format.

# SCHEMAS  
Follow below specification:  

{definition}  

# INPUT


In [4]:
class User(BaseModel):
    """This is a user model that contains name and age."""
    name: str = Field(..., description="the name of the user")
    age: int = Field(..., description="the age of the user")

In [5]:
user_model_definition = get_yaml_schema_definition(User, as_array=True)
print(user_model_definition)

Model: User
Description: This is a user model that contains name and age.

Parameters:
- name (string) # the name of the user
- age (integer) # the age of the user

Return only the YAML format:
```yaml
- name: "<value>"
  age: <value>
- name: "<value>"
  age: <value>
```


In [6]:
so_prompt.from_dict({"definition": user_model_definition})
print(so_prompt.str)

# Information Extractor  

# Persona  
You're an expert on extracting information from given INPUT.  

# Instruction  
- read INPUT carefully and extract information based on SCHEMAS.  

# Caution
- return only specified schemas.  
- no conversation or chit-chat is needed  

# Data Format  
Return the extracted information in YAML format.

# SCHEMAS  
Follow below specification:  

Model: User
Description: This is a user model that contains name and age.

Parameters:
- name (string) # the name of the user
- age (integer) # the age of the user

Return only the YAML format:
```yaml
- name: "<value>"
  age: <value>
- name: "<value>"
  age: <value>
```  

# INPUT


In [7]:
response = model.run(system_prompt=so_prompt.str, messages=[model.UserMessage("Jake is 30 years old, but Kate is 5 year younger than him.")])
print(response)

```yaml
- name: Jake
  age: 30
- name: Kate
  age: 25
```


In [8]:
_response = parse_codeblock_to_dict(response)
[User(**item) for item in _response]

[User(name='Jake', age=30), User(name='Kate', age=25)]

In [9]:
from typing import Literal
class Sentiment(BaseModel):
    """This is a sentiment model that extract sentiment from INPUT with its explanation."""
    sentiment: Literal["positive", "neutral", "negative"] = Field(..., description="read INPUT and classify the sentiment based on options: positive, neutral, neagative")
    evidence: str = Field(..., description="provide evidence from INPUT to support the sentiment")
    explanation: str = Field(..., description="what is the reason for the sentiment")

sentiment_model_definition = get_yaml_schema_definition(Sentiment, as_array=False)
print(sentiment_model_definition)

Model: Sentiment
Description: This is a sentiment model that extract sentiment from INPUT with its explanation.

Parameters:
- sentiment (string) # read INPUT and classify the sentiment based on options: positive, neutral, neagative
- evidence (string) # provide evidence from INPUT to support the sentiment
- explanation (string) # what is the reason for the sentiment

Return only the YAML format:
```yaml
sentiment: "<value>"
evidence: "<value>"
explanation: "<value>"
```


In [10]:
so_prompt.from_dict({"definition": sentiment_model_definition})
print(so_prompt.str)

# Information Extractor  

# Persona  
You're an expert on extracting information from given INPUT.  

# Instruction  
- read INPUT carefully and extract information based on SCHEMAS.  

# Caution
- return only specified schemas.  
- no conversation or chit-chat is needed  

# Data Format  
Return the extracted information in YAML format.

# SCHEMAS  
Follow below specification:  

Model: Sentiment
Description: This is a sentiment model that extract sentiment from INPUT with its explanation.

Parameters:
- sentiment (string) # read INPUT and classify the sentiment based on options: positive, neutral, neagative
- evidence (string) # provide evidence from INPUT to support the sentiment
- explanation (string) # what is the reason for the sentiment

Return only the YAML format:
```yaml
sentiment: "<value>"
evidence: "<value>"
explanation: "<value>"
```  

# INPUT


In [19]:
response = model.run(system_prompt=so_prompt.str, messages=[model.UserMessage("What a wonderful day! I love the sunshine and the birds are singing.")])
print(response)

```yaml
sentiment: "positive"
evidence: "I love the sunshine and the birds are singing"
explanation: "The presence of positive emotions such as love and the description of pleasant natural sounds like birds singing contribute to a positive sentiment."
```


In [20]:
_response = parse_codeblock_to_dict(response)
Sentiment(**_response)

Sentiment(sentiment='positive', evidence='I love the sunshine and the birds are singing', explanation='The presence of positive emotions such as love and the description of pleasant natural sounds like birds singing contribute to a positive sentiment.')